PART1:Warehouse Distance Matrix: Simulates distances between warehouses and customers.
Product Data: Generates synthetic data for products, including pricing, stock levels, and warehouse assignments.
Customer Data: Lists customer names for use in other parts of the system.
This code is primarily for generating initial synthetic datasets, which can be used for further analysis, testing, or system development.

In [ ]:
import pandas as pd
import numpy as np
from faker import Faker

# Initialize Faker instance
fake = Faker()

# Parameters
num_products = 50
num_warehouses = 5
num_customers = 20

# Generate warehouse names
warehouses = [fake.company() for _ in range(num_warehouses)]

# Generate distances between warehouses and customer locations
distance_matrix = pd.DataFrame(
    np.random.randint(50, 1000, size=(num_warehouses, num_customers)),
    index=warehouses,
    columns=[f"Customer {i+1}" for i in range(num_customers)]
)
np.fill_diagonal(distance_matrix.values, 0)
distance_matrix.to_csv('warehouse_distance_matrix.csv')

# Generate product data
products = [fake.unique.word().capitalize() for _ in range(num_products)]
categories = [fake.word().capitalize() for _ in range(num_products)]

product_df = pd.DataFrame({
    'Product ID': [fake.uuid4() for _ in range(num_products)],
    'Product Name': products,
    'Category': categories,
    'Unit Price (₹)': np.round(np.random.uniform(50.0, 1500.0, size=num_products), 2),
    'Stock Level': np.random.randint(0, 1000, size=num_products),
    'Warehouse': np.random.choice(warehouses, size=num_products)
})
product_df.to_csv('updated_synthetic_product_data_with_20_customers.csv', index=False)

# Generate customer data
customers = [f"Customer {i+1}" for i in range(num_customers)]
pd.DataFrame({'Customer': customers}).to_csv('customers.csv', index=False)

print("Initial datasets generated successfully.")


DATA LOADING SCRIPT AS FAKER GENERATES DUPLICATE DATA EVERY TIME

In [2]:
import pandas as pd
import numpy as np
from faker import Faker
import ipywidgets as widgets
from IPython.display import display

# Initialize Faker instance
fake = Faker()

# Load datasets
product_df = pd.read_csv('updated_synthetic_product_data_with_20_customers.csv')
distance_matrix = pd.read_csv('warehouse_distance_matrix.csv', index_col=0)
customers_df = pd.read_csv('customers.csv')
customers = customers_df['Customer'].tolist()

# Load or initialize order data
order_file = 'updated_synthetic_order_data_with_20_customers.csv'
try:
    order_df = pd.read_csv(order_file)
except FileNotFoundError:
    order_df = pd.DataFrame(columns=['Order ID', 'Customer', 'Product Name', 'Quantity', 'Warehouse', 'Total Cost'])

# Load or initialize return data
return_file = 'return_records.csv'
try:
    return_df = pd.read_csv(return_file)
except FileNotFoundError:
    return_df = pd.DataFrame(columns=['Customer', 'Product Name', 'Warehouse', 'Quantity Returned'])

# Display stock levels function
def display_stock_levels():
    print("\nCurrent Stock Levels:")
    for warehouse in product_df['Warehouse'].unique():
        print(f"\nWarehouse: {warehouse}")
        warehouse_stock = product_df[product_df['Warehouse'] == warehouse]
        for _, row in warehouse_stock.iterrows():
            print(f"Product: {row['Product Name']}, Stock Level: {row['Stock Level']}")
    print("\n")

# Display options
def display_options():
    print("Available Customers:")
    for i, customer in enumerate(customers, 1):
        print(f"{i}. {customer}")

    print("\nAvailable Products:")
    for i, product in enumerate(product_df['Product Name'].unique(), 1):
        print(f"{i}. {product}")

# Function to process customer order
def process_order(customer_location, order_items):
    global order_df
    total_cost = 0
    order_records = []
    for item_name, quantity in order_items.items():
        item_info = product_df[product_df['Product Name'] == item_name]
        if item_info.empty:
            print(f"Product {item_name} not found.")
            continue
        
        available_stock = item_info['Stock Level'].values[0]
        assigned_warehouse = item_info['Warehouse'].values[0]
        
        if available_stock >= quantity:
            customer_idx = customers.index(customer_location)
            distances = distance_matrix.loc[assigned_warehouse]
            min_distance = distances[customer_idx]
            transport_cost = min_distance * quantity * 2
            
            # Update stock level
            product_df.loc[product_df['Product Name'] == item_name, 'Stock Level'] -= quantity
            
            # Record the order
            order_id = fake.uuid4()
            order_records.append({
                'Order ID': order_id,
                'Customer': customer_location,
                'Product Name': item_name,
                'Quantity': quantity,
                'Warehouse': assigned_warehouse,
                'Total Cost': transport_cost
            })
            
            total_cost += transport_cost
            print(f"Order for {quantity} units of {item_name} will be fulfilled by {assigned_warehouse}.")
            print(f"Transport cost: ₹{transport_cost:.2f}")
        else:
            print(f"Insufficient stock for {item_name}. Only {available_stock} units available.")
    
    print(f"Total transportation cost for this order: ₹{total_cost:.2f}\n")
    
    # Save the order data
    if order_records:
        new_orders_df = pd.DataFrame(order_records)
        order_df = pd.concat([order_df, new_orders_df], ignore_index=True)
        order_df.to_csv(order_file, index=False)
    
    # Update the product CSV
    product_df.to_csv('updated_synthetic_product_data_with_20_customers.csv', index=False)

# Function to process product return
def process_return(customer_location, return_items):
    global order_df, return_df
    for item_name, return_quantity in return_items.items():
        item_info = product_df[product_df['Product Name'] == item_name]
        if item_info.empty:
            print(f"Product {item_name} not found.")
            continue
        
        # Check if customer has previously ordered this product
        customer_orders = order_df[(order_df['Customer'] == customer_location) & (order_df['Product Name'] == item_name)]
        if customer_orders.empty:
            print(f"Customer {customer_location} has not ordered {item_name}. Cannot process return.")
            continue
        
        # Calculate total ordered quantity for the product by the customer
        total_ordered_quantity = customer_orders['Quantity'].sum()
        if return_quantity > total_ordered_quantity:
            print(f"Return quantity {return_quantity} exceeds the ordered quantity {total_ordered_quantity} for product {item_name}.")
            continue
        
        assigned_warehouse = item_info['Warehouse'].values[0]
        unit_price = item_info['Unit Price (₹)'].values[0]
        
        # Adjust the return quantity to match the total ordered quantity if needed
        original_return_quantity = return_quantity
        
        # Process return for each order related to this product
        for idx, order_row in customer_orders.iterrows():
            if return_quantity <= 0:
                break
            
            if order_row['Quantity'] > return_quantity:
                # Reduce quantity and cost in the existing order
                order_df.loc[idx, 'Quantity'] -= return_quantity
                order_df.loc[idx, 'Total Cost'] -= (return_quantity * unit_price)
                return_quantity = 0
            else:
                # Fully return this order's quantity and reduce the return_quantity accordingly
                return_quantity -= order_row['Quantity']
                order_df.loc[idx, 'Total Cost'] -= (order_row['Quantity'] * unit_price)
                order_df.loc[idx, 'Quantity'] = 0

        # Remove any orders where the quantity has been reduced to zero
        order_df = order_df[order_df['Quantity'] > 0]

        # Update stock level
        product_df.loc[product_df['Product Name'] == item_name, 'Stock Level'] += original_return_quantity
        
        # Record the return
        return_records = {
            'Customer': customer_location,
            'Product Name': item_name,
            'Warehouse': assigned_warehouse,
            'Quantity Returned': original_return_quantity
        }
        return_df = pd.concat([return_df, pd.DataFrame([return_records])], ignore_index=True)
        return_df.to_csv(return_file, index=False)
        
        print(f"Return for {original_return_quantity} units of {item_name} has been processed and added to {assigned_warehouse}.")
    
    print("\nStock Levels After Processing Return:")
    display_stock_levels()
    
    # Update the product CSV and order CSV
    product_df.to_csv('updated_synthetic_product_data_with_20_customers.csv', index=False)
    order_df.to_csv(order_file, index=False)

# Display options
display_options()

# Display initial stock levels
display_stock_levels()

# Define widgets for order and return input
customer_selector = widgets.Dropdown(
    options=[(f"Customer {i+1}", f"Customer {i+1}") for i in range(len(customers))],
    description='Customer:'
)
product_input = widgets.Text(
    description='Product:'
)
quantity_input = widgets.IntText(
    description='Quantity:'
)
submit_button_order = widgets.Button(description="Submit Order")
submit_button_return = widgets.Button(description="Submit Return")

# Display widgets for order and return
display(customer_selector, product_input, quantity_input, submit_button_order, submit_button_return)

# Define button click event handler for order
def on_order_button_click(b):
    customer_location = customer_selector.value
    product_name = product_input.value
    quantity = quantity_input.value
    if product_name not in product_df['Product Name'].values:
        print(f"Product {product_name} not found. Please try again.")
    else:
        order_items = {product_name: quantity}
        # Display stock levels before processing
        print("\nStock Levels Before Processing Order:")
        display_stock_levels()
        # Process the order
        process_order(customer_location, order_items)
        # Display stock levels after processing
        print("\nStock Levels After Processing Order:")
        display_stock_levels()

# Define button click event handler for return
def on_return_button_click(b):
    customer_location = customer_selector.value
    product_name = product_input.value
    quantity = quantity_input.value
    if product_name not in product_df['Product Name'].values:
        print(f"Product {product_name} not found. Please try again.")
    else:
        return_items = {product_name: quantity}
        # Display stock levels before processing
        print("\nStock Levels Before Processing Return:")
        display_stock_levels()
        # Process the return
        process_return(customer_location, return_items)

# Bind button click events to handler functions
submit_button_order.on_click(on_order_button_click)
submit_button_return.on_click(on_return_button_click)


Available Customers:
1. Customer 1
2. Customer 2
3. Customer 3
4. Customer 4
5. Customer 5
6. Customer 6
7. Customer 7
8. Customer 8
9. Customer 9
10. Customer 10
11. Customer 11
12. Customer 12
13. Customer 13
14. Customer 14
15. Customer 15
16. Customer 16
17. Customer 17
18. Customer 18
19. Customer 19
20. Customer 20

Available Products:
1. Recognize
2. Consider
3. Memory
4. Build
5. Commercial
6. Resource
7. News
8. Strategy
9. Performance
10. Professor
11. Worker
12. Stuff
13. Room
14. Front
15. Wall
16. Image
17. Color
18. Price
19. Require
20. Board
21. Free
22. Policy
23. Relationship
24. Avoid
25. Green
26. Around
27. Old
28. Stand
29. Officer
30. Her
31. Late
32. Pass
33. Always
34. Growth
35. Rock
36. Office
37. Country
38. Camera
39. Last
40. Wrong
41. Begin
42. Open
43. Involve
44. Eye
45. Somebody
46. Option
47. Ready
48. Enjoy
49. Character
50. Able

Current Stock Levels:

Warehouse: Hanna LLC
Product: Recognize, Stock Level: 762
Product: Commercial, Stock Level: 663
Pr

Dropdown(description='Customer:', options=(('Customer 1', 'Customer 1'), ('Customer 2', 'Customer 2'), ('Custo…

Text(value='', description='Product:')

IntText(value=0, description='Quantity:')

Button(description='Submit Order', style=ButtonStyle())

Button(description='Submit Return', style=ButtonStyle())


Stock Levels Before Processing Order:

Current Stock Levels:

Warehouse: Hanna LLC
Product: Recognize, Stock Level: 762
Product: Commercial, Stock Level: 663
Product: Strategy, Stock Level: 481
Product: Performance, Stock Level: 256
Product: Worker, Stock Level: 608
Product: Stuff, Stock Level: 846
Product: Front, Stock Level: 630
Product: Board, Stock Level: 50
Product: Free, Stock Level: 881
Product: Avoid, Stock Level: 108
Product: Old, Stock Level: 504
Product: Stand, Stock Level: 904
Product: Her, Stock Level: 160
Product: Growth, Stock Level: 207
Product: Begin, Stock Level: 259
Product: Open, Stock Level: 690
Product: Eye, Stock Level: 194
Product: Ready, Stock Level: 125
Product: Able, Stock Level: 150

Warehouse: Lowe Ltd
Product: Consider, Stock Level: 558
Product: Room, Stock Level: 576
Product: Wall, Stock Level: 67
Product: Image, Stock Level: 621
Product: Color, Stock Level: 668
Product: Relationship, Stock Level: 242
Product: Green, Stock Level: 960
Product: Camera, Sto

C:\Users\PALLAVI\AppData\Local\Temp\ipykernel_2176\2712329243.py:67: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  min_distance = distances[customer_idx]


TASK-The code imports necessary libraries, loads product and forecast data, merges them, and then calculates whether to increase, decrease, or maintain the stock levels based on forecasted demand.
The suggestions for stock changes are displayed, and there is an option to save these suggestions to a file.
Finally, the code calculates the specific changes needed to meet the forecasted demand and prints out a summary of these changes.

In [ ]:
# Cell 1: Import Libraries and Load Data
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load datasets
product_df = pd.read_csv('updated_synthetic_product_data_with_20_customers.csv')

# Cell 2: Create and Save Forecast Data
# Create sample forecast data and save it
forecast_data = pd.DataFrame({
    'Product Name': product_df['Product Name'].unique(),  # Use existing product names
    'Forecasted Demand': np.random.randint(50, 200, size=len(product_df['Product Name'].unique()))  # Random example values
})
forecast_data.to_csv('forecast_data.csv', index=False)

# Load forecast data
forecast_data = pd.read_csv('forecast_data.csv')

# Cell 3: Merge Forecast Data with Current Inventory
# Merge forecast data with current inventory
merged_data = pd.merge(product_df, forecast_data, on='Product Name', how='left')

# Cell 4: Calculate Stock Level Suggestions
# Function to calculate suggested stock levels
def calculate_suggestions(df):
    def update_inventory(row):
        if pd.isna(row['Forecasted Demand']):
            return row['Stock Level'], 'No suggestion'
        
        forecasted_demand = row['Forecasted Demand']
        current_stock = row['Stock Level']
        
        if forecasted_demand > current_stock:
            return forecasted_demand, 'Increase'
        elif forecasted_demand < current_stock:
            return forecasted_demand, 'Decrease'
        else:
            return current_stock, 'No change'
    
    df[['Suggested Stock Level', 'Action']] = df.apply(update_inventory, axis=1, result_type='expand')
    return df

# Cell 5: Display Suggestions and Save to CSV
# Function to display suggestions
def display_suggestions():
    global merged_data
    
    # Calculate suggestions
    suggestions_df = calculate_suggestions(merged_data.copy())
    
    # Display suggestions
    clear_output(wait=True)
    print("Inventory Stock Level Suggestions:")
    display(suggestions_df)
    
    # Add confirmation button
    confirm_button = widgets.Button(description="Confirm and Save")
    display(confirm_button)
    
    def on_confirm_button_click(b):
        global merged_data
        new_file_name = 'updated_inventory_suggestions.csv'
        suggestions_df.to_csv(new_file_name, index=False)
        print(f"\nUpdated stock levels saved to {new_file_name}")
        clear_output(wait=True)
        print("Inventory stock levels analysis and suggestions have been saved.")
    
    confirm_button.on_click(on_confirm_button_click)

# Create button to perform stock level analysis
analyze_button = widgets.Button(description="Analyze Stock Levels")
display(analyze_button)

analyze_button.on_click(lambda b: display_suggestions())

# Cell 6: Calculate Stock Level Changes
# Calculate stock level changes
merged_data['Stock Level Change'] = merged_data['Forecasted Demand'] - merged_data['Stock Level']

# Separate into increases and decreases
stock_changes = merged_data[['Product Name', 'Stock Level Change']]
stock_changes['Action'] = np.where(stock_changes['Stock Level Change'] > 0, 'Increase', 'Decrease')
stock_changes['New Stock Level'] = merged_data['Stock Level'] + merged_data['Stock Level Change']

print("\nSuggested Stock Level Changes:")
print(stock_changes)


Inventory Stock Level Suggestions:


,Product ID,Product Name,Category,Unit Price (₹),Stock Level,Warehouse,Forecasted Demand,Stock Level Change,Suggested Stock Level,Action
0,7217e8f7-5fde-40e1-8f82-9aa65b52dc7f,Recognize,Its,78.04,762,Hanna LLC,82,-680,82,Decrease
1,fe4dfe6d-2423-4e3e-8b08-c10aad304969,Consider,Staff,589.10,558,Lowe Ltd,54,-504,54,Decrease
2,4791c5d5-ff89-470d-8d23-7cb7620dd9ed,Memory,Growth,585.01,108,Farley LLC,142,34,142,Increase
3,ed4a5a87-75fa-4c49-bd62-f16bbf3240eb,Build,Animal,1061.60,319,Fischer-Simon,110,-209,110,Decrease
4,0a4529b8-b4df-4ced-907f-ac5144815f7f,Commercial,Prevent,1340.41,663,Hanna LLC,115,-548,115,Decrease
5,e534753d-1aac-4c02-98b4-5975a3e55b85,Resource,Off,854.73,396,Fischer-Simon,103,-293,103,Decrease
6,f8555c1e-db90-48f9-8f1b-e2948341f331,News,Whom,209.87,636,Morton-Allen,182,-454,182,Decrease
7,192e3d9c-e7bd-45cb-889a-1d76b1c4beec,Strategy,Edge,1018.64,481,Hanna LLC,181,-300,181,Decrease
8,c3955bcc-304a-49d3-af64-4f8b02c8aca4,Performance,Cut,351.69,256,Hanna LLC,174,-82,174,Decrease
9,70cd4903-ac07-42bb-b71f-3ce979ecd9d4,Professor,Office,483.72,275,Morton-Allen,176,-99,176,Decrease


Button(description='Confirm and Save', style=ButtonStyle())

TASK-This script automates the process of analyzing inventory needs, suggesting stock level changes, and performing stock transfers between warehouses.

In [4]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load datasets
product_df = pd.read_csv('updated_synthetic_product_data_with_20_customers.csv')
order_df = pd.read_csv('updated_synthetic_order_data_with_20_customers.csv')
forecast_data = pd.read_csv('forecast_data.csv')  # Forecasted demand data

# Analyze historical sales data
# Aggregating sales data by product and warehouse
historical_sales = order_df.groupby(['Product Name', 'Warehouse'])['Quantity'].sum().reset_index()
historical_sales.rename(columns={'Quantity': 'Historical Sales'}, inplace=True)

# Merge historical sales with current inventory
product_data = pd.merge(product_df, historical_sales, on='Product Name', how='left')

# Merge forecast data with product data
merged_data = pd.merge(product_data, forecast_data, on='Product Name', how='left')

# Function to calculate suggested stock levels
def calculate_suggestions(df):
    def update_inventory(row):
        if pd.isna(row['Forecasted Demand']):
            return row['Stock Level'], 'No suggestion'
        
        forecasted_demand = row['Forecasted Demand']
        historical_sales = row['Historical Sales'] if not pd.isna(row['Historical Sales']) else 0
        current_stock = row['Stock Level']
        
        # Determine action based on forecasted demand and historical sales
        if forecasted_demand > current_stock:
            return forecasted_demand, 'Increase'
        elif forecasted_demand < current_stock:
            return forecasted_demand, 'Decrease'
        else:
            return current_stock, 'No change'
    
    df[['Suggested Stock Level', 'Action']] = df.apply(update_inventory, axis=1, result_type='expand')
    return df

# Calculate suggestions
suggestions_df = calculate_suggestions(merged_data.copy())

# Function to determine stock transfers
def determine_stock_transfers(df):
    transfers = {}
    warehouses = df['Warehouse_x'].unique()
    for _, row in df.iterrows():
        if row['Action'] == 'Increase':
            suggested_quantity = row['Suggested Stock Level'] - row['Stock Level']
            transfers[row['Product Name']] = suggested_quantity
    return transfers

# Function to perform stock transfer
def transfer_stock(transfers):
    global product_df
    
    # Process each transfer
    transfer_records = []
    for product_name, transfer_quantity in transfers.items():
        source_stock = product_df[product_df['Product Name'] == product_name]
        
        for index, row in source_stock.iterrows():
            source_warehouse = row['Warehouse']
            destination_warehouse = 'White Inc'  # Assuming destination warehouse for simplification
            
            source_stock_level = row['Stock Level']
            if source_stock_level >= transfer_quantity:
                product_df.loc[(product_df['Warehouse'] == source_warehouse) & (product_df['Product Name'] == product_name), 'Stock Level'] -= transfer_quantity
                product_df.loc[(product_df['Warehouse'] == destination_warehouse) & (product_df['Product Name'] == product_name), 'Stock Level'] += transfer_quantity
                transfer_records.append((product_name, transfer_quantity, source_warehouse, destination_warehouse))
                break
            else:
                print(f"Insufficient stock for '{product_name}' in '{source_warehouse}'. Only {source_stock_level} units available.")
                
    # Save the updated product data
    product_df.to_csv('updated_synthetic_product_data_with_20_customers.csv', index=False)
    
    # Log transfer details
    transfer_log = pd.DataFrame(transfer_records, columns=['Product Name', 'Quantity Transferred', 'From Warehouse', 'To Warehouse'])
    transfer_log.to_csv('stock_transfer_log.csv', index=False)
    
    print("\nStock levels updated and saved.")
    print("\nStock Transfer Details:")
    print(transfer_log)

# Define widgets
analyze_button = widgets.Button(description="Analyze and Transfer Stock")
output = widgets.Output()

# Display widgets
display(analyze_button, output)

# Define button click event handler for analysis and transfer
def on_analyze_button_click(b):
    with output:
        clear_output()
        
        # Calculate suggestions and determine stock transfers
        suggestions_df = calculate_suggestions(merged_data.copy())
        stock_transfers = determine_stock_transfers(suggestions_df)
        
        if not stock_transfers:
            print("No stock transfers needed based on current suggestions.")
        else:
            print("\nSuggested Stock Transfers:")
            print(stock_transfers)
            
            # Perform stock transfers
            transfer_stock(stock_transfers)

# Bind button click events to handler functions
analyze_button.on_click(on_analyze_button_click)


Button(description='Analyze and Transfer Stock', style=ButtonStyle())

Output()